# F1 Dataset Analysis


In [1]:
import pandas as pd
import plotly.express as px
from pathlib import Path
import plotly.graph_objects as go
import numpy as np
import re

In [2]:
data_path = Path('../data/processed/xgboost/enhanced_f1_dataset.csv')
df = pd.read_csv(data_path)

In [3]:
pd.set_option("display.max_rows", None)
df.head(5)

,year,round,circuit_id,circuit_name,country,date,driver_id,driver_first_name,driver_last_name,driver_nationality,...,avg_speed_rank_in_race,grid_to_finish_change,position_gained,position_lost,has_fastf1_data,group_key,target_position,rank,race_size,position_percentile
0,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,vettel,Sebastian,Vettel,German,...,9.0,2.0,2.0,0.0,True,2018_1,1,1,20,0.05
1,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,hamilton,Lewis,Hamilton,British,...,20.0,-1.0,0.0,1.0,True,2018_1,2,2,20,0.10
2,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,raikkonen,Kimi,Räikkönen,Finnish,...,13.0,-1.0,0.0,1.0,True,2018_1,3,3,20,0.15
3,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,ricciardo,Daniel,Ricciardo,Australian,...,6.0,4.0,4.0,0.0,True,2018_1,4,4,20,0.20
4,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,alonso,Fernando,Alonso,Spanish,...,18.5,5.0,5.0,0.0,True,2018_1,5,5,20,0.25


In [4]:
df = df.drop('is_2018_2021_era', axis=1)

In [5]:

df["driver_full_name"] = df["driver_first_name"] + " " + df["driver_last_name"]

In [6]:

drivers = df[['driver_id', 'driver_first_name', 'driver_last_name', "driver_full_name", 'driver_nationality', 'driver_abbreviation', 'driver_number']].drop_duplicates()
drivers.head(5)

,driver_id,driver_first_name,driver_last_name,driver_full_name,driver_nationality,driver_abbreviation,driver_number
0,vettel,Sebastian,Vettel,Sebastian Vettel,German,VET,5.0
1,hamilton,Lewis,Hamilton,Lewis Hamilton,British,HAM,44.0
2,raikkonen,Kimi,Räikkönen,Kimi Räikkönen,Finnish,RAI,7.0
3,ricciardo,Daniel,Ricciardo,Daniel Ricciardo,Australian,RIC,3.0
4,alonso,Fernando,Alonso,Fernando Alonso,Spanish,ALO,14.0


In [7]:
extra_df = pd.read_csv('../data/extended/f1_driver_data.csv')
extra_df.head(5)

,driver_full_name,typology,enneagram,heaight,why_choose_number,why_choose_number_categorical
0,Sebastian Vettel,INFJ,9w1,175,Chose 5 from karting success; last used by him...,Karting Success
1,Lewis Hamilton,ISFP,3w4,174,From his dad's car number plate F44 when he st...,Personal Connection
2,Kimi Räikkönen,ISTP,5w4,175,Karting number; kept it for career.,Karting Success
3,Daniel Ricciardo,ENFP,7w6,179,Half of his favorite number 33 (birth year 198...,Karting Success
4,Fernando Alonso,ENTJ,1w2,171,"At age 14 on July 14, won karting world champi...",Karting Success


In [8]:
merged = df.merge(
    extra_df,
    on="driver_full_name",   
    how="left"               
)

merged.head()

,year,round,circuit_id,circuit_name,country,date,driver_id,driver_first_name,driver_last_name,driver_nationality,...,target_position,rank,race_size,position_percentile,driver_full_name,typology,enneagram,heaight,why_choose_number,why_choose_number_categorical
0,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,vettel,Sebastian,Vettel,German,...,1,1,20,0.05,Sebastian Vettel,INFJ,9w1,175,Chose 5 from karting success; last used by him...,Karting Success
1,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,hamilton,Lewis,Hamilton,British,...,2,2,20,0.10,Lewis Hamilton,ISFP,3w4,174,From his dad's car number plate F44 when he st...,Personal Connection
2,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,raikkonen,Kimi,Räikkönen,Finnish,...,3,3,20,0.15,Kimi Räikkönen,ISTP,5w4,175,Karting number; kept it for career.,Karting Success
3,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,ricciardo,Daniel,Ricciardo,Australian,...,4,4,20,0.20,Daniel Ricciardo,ENFP,7w6,179,Half of his favorite number 33 (birth year 198...,Karting Success
4,2018,1,albert_park,Albert Park Grand Prix Circuit,Australia,2018-03-25,alonso,Fernando,Alonso,Spanish,...,5,5,20,0.25,Fernando Alonso,ENTJ,1w2,171,"At age 14 on July 14, won karting world champi...",Karting Success


In [9]:
# Driver count by typology from extra_df
typology_counts = extra_df['typology'].value_counts().sort_values(ascending=True)

fig = px.bar(
    x=typology_counts.values,
    y=typology_counts.index,
    orientation='h',
    title='Unique Driver Count by MBTI Personality Type',
    labels={'x': 'Number of Drivers', 'y': 'MBTI Type'},
    text=typology_counts.values
)

fig.update_layout(
    width=800,
    height=600,
    xaxis_title="Number of Drivers",
    yaxis_title="MBTI Personality Type"
)

fig.update_traces(textposition='outside')
fig.show()

print("Driver distribution by personality type:")
for mbti_type, count in typology_counts.sort_values(ascending=False).items():
    print(f"{mbti_type}: {count} drivers")
    
print(f"\nTotal drivers with typology data: {len(extra_df)}")

Driver distribution by personality type:
ISTP: 9 drivers
ISFP: 4 drivers
INFP: 4 drivers
ENFP: 3 drivers
ISFJ: 3 drivers
ESFP: 3 drivers
INFJ: 2 drivers
ENTJ: 2 drivers
ENFJ: 2 drivers
INTJ: 1 drivers
ENTP: 1 drivers
ESFJ: 1 drivers
ESTJ: 1 drivers
ESTP: 1 drivers

Total drivers with typology data: 43


In [10]:
column_analysis = {
    'ERGAST API - BASIC RACE DATA': {
        'columns': [
            'year', 'round', 'race_name', 'date', 'circuit_id', 'circuit_name', 'country',
            'driver_id', 'driver_first_name', 'driver_last_name', 'driver_nationality',
            'constructor_id', 'constructor_name', 'constructor_nationality',
            'grid_position', 'position', 'target_position'
        ],
        'source': 'Ergast API',
        'type': 'Raw API Data',
        'description': 'Basic F1 race information, results, driver/team details'
    },
    
    'ERGAST FEATURE ENGINEERED': {
        'columns': [
            'group_key', 'rank', 'race_size', 'position_percentile',
            'driver_career_wins', 'driver_career_podiums', 'driver_career_total_points',
            'driver_career_avg_points', 'driver_career_avg_position', 'driver_career_race_count',
            'driver_win_rate', 'driver_podium_rate', 'driver_points_rate',
            'constructor_career_wins', 'constructor_career_podiums', 'constructor_career_total_points',
            'constructor_career_avg_points', 'constructor_career_avg_position', 'constructor_career_race_count',
            'constructor_win_rate', 'constructor_podium_rate', 'constructor_points_rate',
            'driver_track_avg_points', 'driver_track_avg_position', 'driver_track_race_count',
            'constructor_track_avg_points', 'constructor_track_avg_position', 'constructor_track_race_count'
        ],
        'source': 'Ergast API (Processed)',
        'type': 'Feature Engineered',
        'description': 'Career statistics, track-specific performance, ratios'
    },
    
    'FASTF1 API - TELEMETRY DATA': {
        'columns': [
            'driver_abbreviation', 'driver_number', 'fastest_lap_time', 'avg_lap_time', 'lap_time_std', 'total_laps',
            'avg_sector1_time', 'avg_sector2_time', 'avg_sector3_time', 'max_speed', 'avg_speed', 'speed_variance',
            'main_compound', 'compound_changes', 'pit_stops', 'is_soft_primary', 'is_hard_primary', 'is_medium_primary',
            'is_intermediate', 'is_wet', 'air_temp', 'track_temp', 'humidity', 'wind_speed'
        ],
        'source': 'FastF1 API',
        'type': 'Raw API Data',
        'description': 'Telemetry data, tire strategy, weather conditions'
    },
    
    'FASTF1 FEATURE ENGINEERED': {
        'columns': [
            'compound_changes_normalized', 'pit_stops_normalized', 'low_pit_strategy', 'aggressive_pit_strategy',
            'speed_consistency', 'speed_efficiency', 'total_sector_time', 'sector1_dominance', 'sector2_dominance',
            'sector3_dominance', 'lap_time_efficiency', 'lap_time_consistency_score', 'temp_differential',
            'is_hot_conditions', 'is_cold_conditions', 'is_high_track_temp', 'is_high_humidity', 'is_low_humidity',
            'is_windy_conditions', 'era_adjusted_speed', 'era_adjusted_laps', 'avg_lap_time_vs_race_avg',
            'avg_lap_time_vs_race_best', 'avg_lap_time_rank_in_race', 'max_speed_vs_race_avg', 'max_speed_vs_race_best',
            'max_speed_rank_in_race', 'avg_speed_vs_race_avg', 'avg_speed_vs_race_best', 'avg_speed_rank_in_race',
            'grid_to_finish_change', 'position_gained', 'position_lost', 'has_fastf1_data'
        ],
        'source': 'FastF1 API (Processed)',
        'type': 'Feature Engineered',
        'description': 'Normalizations, comparisons, boolean flags'
    },
    
    'ERA AND REGULATION FLAGS': {
        'columns': [
            'is_2017_plus_era', 'is_2018_2021_era', 'is_2022_plus_era', 'is_2022_plus_era_fastf1',
            'is_covid_season_2020', 'is_covid_season_2020_fastf1', 'has_sprint_format', 'has_sprint_format_fastf1'
        ],
        'source': 'Multiple Sources',
        'type': 'Feature Engineered',
        'description': 'Regulation periods, special formats'
    }
}

# Appendix DataFrame oluştur
appendix_data = []

for category, info in column_analysis.items():
    found_columns = [col for col in info['columns'] if col in merged.columns]
    
    for col in found_columns:
        
        missing_count = merged[col].isnull().sum()
        missing_percentage = (missing_count / len(merged)) * 100
        data_type = str(merged[col].dtype)
        
        appendix_data.append({
            'Category': category,
            'Column': col,
            'Source': info['source'],
            'Type': info['type'],
            'Description': info['description'],
            'Data_Type': data_type,
            'Missing_Count': missing_count,
            'Missing_Percentage': round(missing_percentage, 1),
            'Completeness_Percentage': round(100 - missing_percentage, 1)
        })

appendix_df = pd.DataFrame(appendix_data)
appendix_df.head()

,Category,Column,Source,Type,Description,Data_Type,Missing_Count,Missing_Percentage,Completeness_Percentage
0,ERGAST API - BASIC RACE DATA,year,Ergast API,Raw API Data,"Basic F1 race information, results, driver/tea...",int64,0,0.0,100.0
1,ERGAST API - BASIC RACE DATA,round,Ergast API,Raw API Data,"Basic F1 race information, results, driver/tea...",int64,0,0.0,100.0
2,ERGAST API - BASIC RACE DATA,date,Ergast API,Raw API Data,"Basic F1 race information, results, driver/tea...",object,0,0.0,100.0
3,ERGAST API - BASIC RACE DATA,circuit_id,Ergast API,Raw API Data,"Basic F1 race information, results, driver/tea...",object,0,0.0,100.0
4,ERGAST API - BASIC RACE DATA,circuit_name,Ergast API,Raw API Data,"Basic F1 race information, results, driver/tea...",object,0,0.0,100.0


In [11]:
appendix_df.groupby(["Category", "Source"])["Missing_Percentage"].median()

Category                      Source                
ERA AND REGULATION FLAGS      Multiple Sources           0.0
ERGAST API - BASIC RACE DATA  Ergast API                 0.0
ERGAST FEATURE ENGINEERED     Ergast API (Processed)     0.0
FASTF1 API - TELEMETRY DATA   FastF1 API                28.7
FASTF1 FEATURE ENGINEERED     FastF1 API (Processed)    30.3
Name: Missing_Percentage, dtype: float64

## **FASTF1 TELEMETRY DATA - Missing Values Analysis**

### **Category Overview:**
- **Total columns:** 24
- **Data source:** FastF1 API (Raw telemetry data)

### **Detailed Analysis:**

| Column Group | Column | Missing % | Status |
|--------------|--------|-----------|--------|
| **Driver Info** | `driver_abbreviation` | 28.7% | 🟡 Moderate |
| | `driver_number` | 28.7% | 🟡 Moderate |
| **Lap & Speed Data** | `fastest_lap_time` | 30.3% | 🟠 High |
| | `avg_lap_time` | 30.3% | 🟠 High |
| | `lap_time_std` | 31.4% | 🟠 High |
| | `total_laps` | 28.7% | 🟡 Moderate |
| | `max_speed` | 30.3% | 🟠 High |
| | `avg_speed` | 30.3% | 🟠 High |
| | `speed_variance` | 30.8% | 🟠 High |
| **Sector Times** | `avg_sector1_time` | 30.8% | 🟠 High |
| | `avg_sector2_time` | 30.3% | 🟠 High |
| | `avg_sector3_time` | 30.3% | 🟠 High |
| **Tire Strategy** | `main_compound` | 28.8% | 🟡 Moderate |
| | `compound_changes` | 28.7% | 🟡 Moderate |
| | `pit_stops` | 28.7% | 🟡 Moderate |
| | `is_soft_primary` | 28.7% | 🟡 Moderate |
| | `is_hard_primary` | 28.7% | 🟡 Moderate |
| | `is_medium_primary` | 28.7% | 🟡 Moderate |
| | `is_intermediate` | 28.7% | 🟡 Moderate |
| | `is_wet` | 28.7% | 🟡 Moderate |

### **Missing Value Distribution:**
- **Moderate (10-30%):** 11 columns  
- **High (30-50%):** 9 columns  

In [12]:
numeric_cols = merged.select_dtypes(include=[np.number]).columns.tolist()

# Correlation matrix
corr_matrix = merged[numeric_cols].corr()

In [13]:
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.8:
            high_corr_pairs.append({
                'Feature1': corr_matrix.columns[i],
                'Feature2': corr_matrix.columns[j],
                'Correlation': round(corr_val, 3)
            })

high_corr_df = pd.DataFrame(high_corr_pairs)
if not high_corr_df.empty:
    high_corr_df = high_corr_df.sort_values('Correlation', key=abs, ascending=False)
    print(f"High correlation(|r| > 0.8):")
    print(high_corr_df.to_string(index=False))


High correlation(|r| > 0.8):
                       Feature1                        Feature2  Correlation
                           rank             position_percentile        1.000
              has_sprint_format        has_sprint_format_fastf1        1.000
        constructor_career_wins            constructor_win_rate        1.000
                       position                 target_position        1.000
                       position             position_percentile        1.000
                       position                            rank        1.000
           is_covid_season_2020     is_covid_season_2020_fastf1        1.000
                target_position                            rank        1.000
               is_2022_plus_era         is_2022_plus_era_fastf1        1.000
                target_position             position_percentile        1.000
                      max_speed              era_adjusted_speed        0.999
     constructor_career_podiums         constru

In [14]:
# Features to drop based on correlation analysis
features_to_drop = [
    # Perfect duplicates (r=1.0) - keep one from each pair
    'position_percentile',  # keep rank instead
    'has_sprint_format_fastf1',  # keep has_sprint_format
    'constructor_win_rate',  # keep constructor_career_wins
    'target_position',  # keep position 
    'is_covid_season_2020_fastf1',  # keep is_covid_season_2020
    'is_2022_plus_era_fastf1',  # keep is_2022_plus_era
    
    # Speed redundancy (r>0.97)
    'era_adjusted_speed',  # keep max_speed and avg_speed
    'era_adjusted_laps',   # keep total_laps
    
    # Career stats redundancy - keep raw counts, drop rates
    'constructor_podium_rate',  # keep constructor_career_podiums
    'driver_win_rate',  # keep driver_career_wins
    'driver_podium_rate',  # keep driver_career_podiums
    'constructor_points_rate',  # keep constructor_career_total_points
    'driver_points_rate',  # keep driver_career_total_points
    
    # Lap time redundancy
    'total_sector_time',  # keep avg_lap_time and fastest_lap_time
    
    # Track performance redundancy 
    'constructor_track_avg_points',  # highly correlated with career stats
    'driver_track_avg_points',  # highly correlated with career stats
    
    # Race comparison redundancy
    'avg_speed_vs_race_best',  # keep avg_speed_vs_race_avg
    'max_speed_vs_race_best',  # keep max_speed_vs_race_avg
    'avg_lap_time_vs_race_best',  # keep avg_lap_time_vs_race_avg
]

print(f"Features to drop: {len(features_to_drop)}")
print(f"Remaining features: {len(numeric_cols) - len(features_to_drop)}")

# Apply the drops
merged_clean = merged.drop(columns=features_to_drop, errors='ignore')
print(f"\nDataset shape after dropping correlated features: {merged_clean.shape}")


Features to drop: 19
Remaining features: 76

Dataset shape after dropping correlated features: (3318, 96)


In [15]:
# Remove ID columns and target leakage columns (KEEP grid_position!)
id_columns = ['driver_id', 'circuit_id', 'constructor_id', 'group_key']
target_leakage = ['rank', 'points', 'status', 'grid_to_finish_change', 'position_gained', 'position_lost']

# Keep grid_position as feature, only remove 'position' as target  
model_features = merged_clean.drop(columns=id_columns + target_leakage + ['position'], errors='ignore')
print(f"Final model dataset: {model_features.shape}")
print(f"Grid position kept: {'grid_position' in model_features.columns}")

Final model dataset: (3318, 87)
Grid position kept: True


In [16]:
# Drop unnecessary categorical columns
drop_categorical = ['date', 'circuit_name', 'driver_first_name', 'driver_last_name', 
                   'driver_full_name', 'constructor_name', 'why_choose_number']

final_model_data = model_features.drop(columns=drop_categorical, errors='ignore')
print(f"Final model dataset: {final_model_data.shape}")


Final model dataset: (3318, 80)


In [28]:
# Save final_model_data to CSV
output_path = '../data/processed/xgboost/final_model_dataset.csv'

try:
    print(f"final_model_data shape: {final_model_data.shape}")
    print(f"final_model_data columns: {len(final_model_data.columns)}")
    print(f"First few columns: {final_model_data.columns[:5].tolist()}")

    final_model_data.to_csv(output_path, index=False)
    print(f"\nSuccessfully saved to: {output_path}")

except NameError:
    print("final_model_data not found - checking other datasets...")
    datasets = ['merged', 'merged_clean', 'model_features', 'data_clean']
    for dataset in datasets:
        try:
            df = eval(dataset)
            print(f"\n{dataset} shape: {df.shape}")
            print(f"{dataset} columns: {len(df.columns)}")
            print(f"First few columns: {df.columns[:5].tolist()}")

            # Save the best available dataset
            output_path = f'../data/processed/xgboost/{dataset}_dataset.csv'
            df.to_csv(output_path, index=False)
            print(f"Saved {dataset} to: {output_path}")
            break
        except:
            continue

except Exception as e:
    print(f"Error: {e}")


final_model_data shape: (3318, 80)
final_model_data columns: 80
First few columns: ['year', 'round', 'country', 'driver_nationality', 'constructor_nationality']

Successfully saved to: ../data/processed/xgboost/final_model_dataset.csv
